In [18]:
# This is a sample Python script.
import pandas as pd
import torch
device = torch.device('cpu')
import dgl
import networkx as nx
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn
import torch.optim as optim
import dgl.function as fn


In [19]:
df=pd.read_csv("../data/PaySim_kaggle.csv")

df=df.sample(n=5000000)

print(df.head())

         step      type     amount     nameOrig  oldbalanceOrg  \
5833298   402   PAYMENT   16663.31   C376362460       68255.17   
740242     38   CASH_IN  366824.91  C1092369823       11653.00   
6032727   474   PAYMENT    3335.39  C1731599343           0.00   
5617036   395  CASH_OUT  284818.10   C613449053       18677.00   
1551687   154  CASH_OUT   78014.47  C2085758934         148.00   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
5833298        51591.86  M1472741442            0.00            0.00        0   
740242        378477.91   C670698587            0.00            0.00        0   
6032727            0.00  M1392062286            0.00            0.00        0   
5617036            0.00   C933740624       935226.32      1220044.42        0   
1551687            0.00  C2130767207            0.00        78014.47        0   

         isFlaggedFraud  
5833298               0  
740242                0  
6032727               0  
5617036     

Adjust data proportions as the fraud data is heavily biased having less fraudulant cases.

In [20]:
# Set the proportion of non-fraud to remove (e.g., 50% of non-fraud instances)
remove_fraction = 0.99

# Separate the fraud and non-fraud instances
fraud_df = df[df['isFraud'] == 1]
non_fraud_df = df[df['isFraud'] == 0]

# Randomly sample and remove 'remove_fraction' proportion of non-fraud instances
non_fraud_to_remove = non_fraud_df.sample(frac=remove_fraction, random_state=42)

# Drop the sampled non-fraud instances from the DataFrame
df = df.drop(non_fraud_to_remove.index)

# Verify the new balance
label_counts = df['isFraud'].value_counts()
fraud_ratio = label_counts[1] / len(df)
non_fraud_ratio = label_counts[0] / len(df)

print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")


Fraud count: 6423
Non-fraud count: 49936
Fraud ratio: 0.1140
Non-fraud ratio: 0.8860


In [21]:
# Create a mapping from unique user names to numeric IDs (nodes)
user_mapping = {user: idx for idx, user in enumerate(set(df['nameOrig']).union(set(df['nameDest'])))}

# Create edges between nameOrig and nameDest
src = df['nameOrig'].map(user_mapping).values
dst = df['nameDest'].map(user_mapping).values

In [22]:
print(src)

[53133 66976 87427 ... 45822 19224 21063]


In [23]:
# Create a DGL graph from the source and destination nodes
g = dgl.graph((src, dst))


In [24]:
# Add transaction amount as edge feature
g.edata['amount'] = torch.tensor(df['amount'].values, dtype=torch.float32)

# Optional: Add fraud information to edge features
g.edata['isFraud'] = torch.tensor(df['isFraud'].values, dtype=torch.float32)

# Initialize node features with zeros (this handles all nodes)
num_nodes = g.num_nodes()
balance_orig = torch.zeros(num_nodes, dtype=torch.float32)
balance_dest = torch.zeros(num_nodes, dtype=torch.float32)

for orig_user, balance in df[['nameOrig', 'oldbalanceOrg']].drop_duplicates().values:
    balance_orig[user_mapping[orig_user]] = balance

for dest_user, balance in df[['nameDest', 'oldbalanceDest']].drop_duplicates().values:
    balance_dest[user_mapping[dest_user]] = balance

node_features = torch.stack([balance_orig, balance_dest], dim=1)  # Changed to stack both features

g.ndata['features'] = node_features


In [25]:
g.edata['isFraud'].shape
#node_features.shape
g.num_edges

<bound method DGLGraph.num_edges of Graph(num_nodes=110593, num_edges=56359,
      ndata_schemes={'features': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={'amount': Scheme(shape=(), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)})>

In [26]:
# Print graph information
#print(g)

# Visualize the graph using NetworkX (convert DGL graph to NetworkX for visualization)
#nx_graph = g.to_networkx()

# Optional: Visualize using a layout for better readability
#pos = nx.spring_layout(nx_graph)  # Use a layout for better visualization
#plt.figure(figsize=(12, 12))
#nx.draw(nx_graph, pos, node_size=50, node_color='skyblue', font_size=10, with_labels=True)
#plt.show()

In [27]:
# Assuming that you have edge features like transaction amounts or fraud status
edge_features = g.edata.get('amount', None)  # Assuming 'amount' is an edge feature
if edge_features is not None:
    print("Edge Features (Transaction Amounts):")
    print(edge_features)
else:
    print("No edge features found.")


Edge Features (Transaction Amounts):
tensor([7.5386e+04, 6.6723e+05, 4.4791e+04,  ..., 3.0052e+05, 4.7404e+02,
        4.1814e+04])


In [28]:
edges = g.edges()
# Split the dataset into train and test set
# Generate indices for train-test split (80% train, 20% test)
num_train_edges = int(0.8 * len(edges[0]))
train_indices = torch.arange(num_train_edges)
test_indices = torch.arange(num_train_edges, len(edges[0]))

# Create masks for training and testing
train_mask = torch.zeros(len(edges[0]), dtype=torch.bool)
test_mask = torch.zeros(len(edges[0]), dtype=torch.bool)
train_mask[train_indices] = 1
test_mask[test_indices] = 1


In [44]:
# Define the GraphSAGE model for fraud detection
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGE, self).__init__()
        self.layer1 = dgl.nn.SAGEConv(in_feats, hidden_feats, 'mean')
        self.layer2 = dgl.nn.SAGEConv(hidden_feats, out_feats, 'mean')
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(out_feats * 2, 1)  # * Concatenate source and destination node embeddings *
        
    def forward(self, g, features):
        # Apply first GraphSAGE layer and ReLU
        x = self.layer1(g, features)
        x = torch.relu(x)
        x = self.dropout(x)

        # Apply second GraphSAGE layer to get embeddings
        x = self.layer2(g, x)
        
        # Get source and destination nodes for each edge
        src, dst = g.edges()  # Get indices of source and destination nodes
        src_embeddings = x[src]  # Embeddings for source nodes
        dst_embeddings = x[dst]  # Embeddings for destination nodes
        
        # Concatenate source and destination node embeddings to create edge features
        edge_features = torch.cat([src_embeddings, dst_embeddings], dim=1)  # Concatenate along the feature dimension
        logits = self.fc(edge_features).squeeze()  # * Output a single value per edge (fraud score) *
        return logits


In [30]:
# Count occurrences of each label in the 'isFraud' column
label_counts = df['isFraud'].value_counts()

# Calculate the proportion of each class
fraud_ratio = label_counts[1] / len(df)  # Assuming '1' represents fraud
non_fraud_ratio = label_counts[0] / len(df)  # Assuming '0' represents non-fraud

# Print the results
print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")

Fraud count: 6423
Non-fraud count: 49936
Fraud ratio: 0.1140
Non-fraud ratio: 0.8860


In [96]:
# Instantiate the model
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
in_feats = 2  # balanceOrig and balanceDest features
hidden_feats = 64
out_feats = 1  # Fraud (binary classification)

pos_weight = torch.tensor([non_fraud_ratio/fraud_ratio],dtype=torch.float)

model = GraphSAGE(in_feats, hidden_feats, out_feats)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()

    # Forward pass
    logits = model(g, g.ndata['features'])
    #print(logits.shape)

    # Get target labels for the fraud detection task
    labels = g.edata['isFraud']
    breakpoint()
    # Compute loss (use train_mask to filter out test edges)
    #print("labels",labels[train_mask].view(-1,1).shape)
    #print("logits", logits[train_mask].view(-1,1).shape)
    loss = loss_fn(logits[train_mask].view(-1,1), labels[train_mask].view(-1,1))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")



Epoch 2/200, Loss: 443286.6875
Epoch 4/200, Loss: 330277.40625
Epoch 6/200, Loss: 240058.8125
Epoch 8/200, Loss: 232801.71875
Epoch 10/200, Loss: 227355.984375
Epoch 12/200, Loss: 212179.6875
Epoch 14/200, Loss: 174467.9375
Epoch 16/200, Loss: 157055.703125
Epoch 18/200, Loss: 123834.6953125
Epoch 20/200, Loss: 100676.6953125
Epoch 22/200, Loss: 91932.4140625
Epoch 24/200, Loss: 90944.578125
Epoch 26/200, Loss: 76480.8203125
Epoch 28/200, Loss: 76917.21875
Epoch 30/200, Loss: 60013.40234375
Epoch 32/200, Loss: 50517.64453125
Epoch 34/200, Loss: 45130.55859375
Epoch 36/200, Loss: 36097.92578125
Epoch 38/200, Loss: 33103.75
Epoch 40/200, Loss: 34758.8671875
Epoch 42/200, Loss: 37220.44921875
Epoch 44/200, Loss: 25694.14453125
Epoch 46/200, Loss: 24046.47265625
Epoch 48/200, Loss: 22496.0625
Epoch 50/200, Loss: 18533.373046875
Epoch 52/200, Loss: 19977.6953125
Epoch 54/200, Loss: 17224.845703125
Epoch 56/200, Loss: 15881.20703125
Epoch 58/200, Loss: 13609.921875
Epoch 60/200, Loss: 15090.

Epoch 2/20, Loss: 50026236.0
Epoch 4/20, Loss: 21910472.0
Epoch 6/20, Loss: 12671362.0
Epoch 8/20, Loss: 12492275.0
Epoch 10/20, Loss: 13373789.0
Epoch 12/20, Loss: 14267830.0
Epoch 14/20, Loss: 14137046.0
Epoch 16/20, Loss: 12949893.0
Epoch 18/20, Loss: 11810090.0
Epoch 20/20, Loss: 10091464.0
0.1
Accuracy: 0.7332
Recall: 0.0086
F1 Score: 0.0073
0.01
Accuracy: 0.7535
Recall: 0.0039
F1 Score: 0.0036
0.2
Accuracy: 0.7378
Recall: 0.0102
F1 Score: 0.0087
0.4
Accuracy: 0.7405
Recall: 0.0079
F1 Score: 0.0068

In [97]:


# Evaluate the model
model.eval()
with torch.no_grad():
    logits = model(g, g.ndata['features'])

    # Convert logits to probabilities
    predictions = torch.sigmoid(logits).squeeze()
    
    # Apply threshold of 0.5 to classify fraud
    predicted_labels = (predictions > 0.).float()
    predicted_labels = predicted_labels[test_mask]  # Apply test_mask here
    # Get actual labels
    #true_labels = g.edata['isFraud']
    true_labels = g.edata['isFraud'][test_mask]  # Apply test_mask here
    # Compute accuracy manually
    correct = (predicted_labels == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = correct / total

    # Print evaluation results
    print(f"Accuracy: {accuracy:.4f}")
    # Calculate F1-Score
    f1 = f1_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    print(f"Recall: {recall:.4f}")
    print(f"Precision:{precision:.4f}")
    print(f"F1 Score: {f1:.4f}")

Accuracy: 0.3674
Recall: 0.9560
Precision:0.1468
F1 Score: 0.2545


In [98]:
pos_weight

tensor([7.7746])

In [51]:
sum(labels==1)

tensor(6423)